## Домашнее задание к лекции.

### Задание 1
**Напишите функцию**, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец **class**

In [1]:
# -- Общее для всех -- #

import pandas as pd 


debug = True # Дебаг

In [3]:
ratings = pd.read_csv('database/ratings.csv')

def classification(data):
    if data <= 2: return 'низкий рейтинг'
    elif data <= 4: return 'средний рейтинг'
    elif (data >= 4.5) and (data <= 5): return 'высокий рейтинг'
    return

ratings['class'] = ratings['rating'].apply(classification)
if debug:
    print(ratings.head())
    print()
    print(ratings['class'].value_counts())
    print()
    print(ratings['class'].value_counts(normalize=True))

   userId  movieId  rating   timestamp            class
0       1       31     2.5  1260759144  средний рейтинг
1       1     1029     3.0  1260759179  средний рейтинг
2       1     1061     3.0  1260759182  средний рейтинг
3       1     1129     2.0  1260759185   низкий рейтинг
4       1     1172     4.0  1260759205  средний рейтинг

средний рейтинг    63801
высокий рейтинг    22818
низкий рейтинг     13385
Name: class, dtype: int64

средний рейтинг    0.637984
высокий рейтинг    0.228171
низкий рейтинг     0.133845
Name: class, dtype: float64


### Задание 2
Используем файл **keywords.csv**

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим **‘undefined’**.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

``` Python
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```

Результат классификации запишите **в отдельный столбец region**.

In [4]:
requests = pd.read_csv('database/keywords.csv')

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

all_geo_data_values = [] # для проверки
for i in geo_data.keys():
    all_geo_data_values += geo_data[i]

def region_function(data):
    flag = False
    keyword_i = data.replace(',','').replace('.','').lower().split()
    for element in keyword_i:
        if element in all_geo_data_values: # Проверка
            flag = True                    # Нашли совпадение города
            finded_keyword = element 
    
    if flag:                               # Будем проверять регион
        for region in geo_data.keys():
            if finded_keyword in geo_data[region]: return region  
                    
    return 'undefined'                     # Ничего не нашел
        
requests['region'] = requests['keyword'].apply(region_function)


print(requests.head())

if debug: 
    print()
    print( requests.groupby('region').count().head())

         keyword     shows     region
0             вк  64292779  undefined
1  одноклассники  63810309  undefined
2          порно  41747114  undefined
3           ютуб  39995567  undefined
4      вконтакте  21014195  undefined

                keyword  shows
region                        
undefined         99406  99406
Дальний Восток       64     64
Северо-Запад        148    148
Центр               382    382


### Задание 3 **(бонусное)**

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную **years** запишите список из всех годов с 1950 по 2010.

1. Напишите функцию **production_year**, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    * для каждой строки пройдите по всем годам списка **years**
    * если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    * если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

1. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец **‘year’**

1. Посчитайте средний рейтинг всех фильмов для каждого значения столбца **‘year’** и отсортируйте результат по убыванию рейтинга

In [44]:
# --- option 1. Hit the years_list, Jack ---# 
movies = pd.read_csv('database/movies.csv')

years = list(range(1950,2011))
#print(years)

def production_year(data):
    splitted_text = data.replace(',',' ').replace('(',' ').replace(')','').split()
    #print(splitted_text)
    for element in splitted_text:
        #print(element)
        if element in str(years):               # Проверка
            if int(element) in years:           # Нашли совпадение года 
                return element                      
                    
    return '1900'                     # Ничего не нашел
    
movies['year'] = movies['title'].apply(production_year)

movies.head() 
movies.groupby('year').count()

,movieId,title,genres
year,,,
1900,1612,1612,1612
1950,27,27,27
1951,23,23,23
1952,27,27,27
1953,36,36,36
...,...,...,...
2006,261,261,261
2007,255,255,255
2008,247,247,247


In [36]:
ratings  = pd.read_csv('database/ratings.csv')

A = ratings.groupby('movieId')['rating'].mean().reset_index()

def mean_rating(data):  
    if len(A[ A['movieId'] == data ]['rating']) != 0: 
        return tuple(A[ A['movieId'] == data ]['rating'])[0]
    return 0

movies['mean_rating'] = movies['movieId'].apply(mean_rating)

B = movies.groupby('year')['mean_rating'].mean()
B.reset_index().sort_values(by = 'mean_rating',ascending = False)

,year,mean_rating
1,1951,3.906653
12,1962,3.772962
23,1973,3.688212
3,1953,3.635020
7,1957,3.603241
...,...,...
50,2000,3.176118
6,1956,3.174243
40,1990,3.165484
53,2003,3.140829


In [66]:
# --- option 2. Use Regex, Luke! ---# 
movies = pd.read_csv('database/movies.csv')

import regex as re

#movies.head()
def production_year(data):
    pattern = r'\((\d{4})\)'
    if re.findall(pattern,data):
        year = int(re.findall(pattern,data)[0])
        if (year > 1950) and (year < 2010):
            return int(re.findall(pattern,data)[0])
    return '1900'                     # Ничего не нашел
    
    
movies['year'] = movies['title'].apply(production_year)

movies.head() 

movies.groupby('year').count().head()


,movieId,title,genres
year,,,
1951,23,23,23
1952,27,27,27
1953,36,36,36
1954,33,33,33
1955,49,49,49


In [67]:
ratings  = pd.read_csv('database/ratings.csv')

A = ratings.groupby('movieId')['rating'].mean().reset_index()

def mean_rating(data):  
    if len(A[ A['movieId'] == data ]['rating']) != 0: 
        return tuple(A[ A['movieId'] == data ]['rating'])[0]
    return 0

movies['mean_rating'] = movies['movieId'].apply(mean_rating)

B = movies.groupby('year')['mean_rating'].mean()
B.reset_index().sort_values(by = 'mean_rating',ascending = False).head()

,year,mean_rating
0,1951,3.906653
11,1962,3.772962
22,1973,3.688212
2,1953,3.635020
6,1957,3.603241


Решение 1 и 2 совпадают, только в решении №2 года можно расширить..

Если можно было как то по-дугому найти средний рейтинг, подскажите, пожалуйста.